In [1]:
%%time
# NEA Link
link = "https://www.nea.gov.sg/dengue-zika/dengue/dengue-clusters"

# Importing Libraries
import requests, pandas as pd, numpy as np, time, glob, os, sys, win32com.client, pythoncom, pickle, json, re
import shapely.speedups, geopandas as gpd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from shapely.geometry import Point
from shapely.geometry import shape

shapely.speedups.enable()
tqdm.pandas()

CPU times: total: 484 ms
Wall time: 514 ms


In [2]:
# Get all snapshots of NEA's dengue cases via wayback machine from 2020 to 2022
def get_snapshots(link):
    # Use Wayback server to get all snapshots of the link
    wayback = "https://web.archive.org/cdx/search/cdx?url=" + link + "&output=json&fl=timestamp,original&collapse=digest&filter=statuscode:200"
    response = requests.get(wayback)
    print(response)
    data = response.json()
    snapshots = data[1:]
    # Keep only snapshots from 2021 to 2022
    snapshots = [snap for snap in snapshots if snap[0][:4] in ["2021", "2022"]]
    # Keep only timestamps
    snapshots = [snap[0] for snap in snapshots]
    return snapshots

In [3]:
# Get the snapshots
while True:
    try: 
        snapshots = get_snapshots(link)
        break
    except: 
        print("Error")
        time.sleep(1)
        continue
snapshots

<Response [200]>


['20210308151403',
 '20210808172247',
 '20211027051137',
 '20211203081908',
 '20220310082530',
 '20220319234337',
 '20220329152026',
 '20220425143805',
 '20220605020619',
 '20220609053540',
 '20220616104319',
 '20220618163415',
 '20220805162654',
 '20220814023504',
 '20220901011605',
 '20220907192246',
 '20220911051635',
 '20220918172033',
 '20220921205322',
 '20221025131215']

In [4]:
def get_soup(link, timestamp):
    # Get the snapshot
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }
    snapshot = "https://web.archive.org/web/" + timestamp + "/" + link
    response = requests.get(snapshot, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    # print(snapshot)
    return soup

In [5]:
def extract_clusters(soup):
    # In the soup, we want to extract the table with the class "dengue-fixed-table"
    table = soup.find_all("table", class_="dengue-fixed-table")[0]

    # The number of clusters is the number of times data-row-table appears
    # We want to extract the number of clusters
    clusters = len(table.find_all("table", {"data-row-table": True}))
    # print("Number of clusters:", clusters, "\n")
    return clusters, table

def extract_cluster(cluster_no, table, snapshot_no):
    # Let's extract the data from the first cluster
    rows = table.find_all("tr", {"data-row": cluster_no})[-1]
    # print("Cluster No.:", cluster_no)

    # the last 2 td tags with a number is the total number of cases and recent cases
    total = int(rows.find_all("td")[-1].text)
    recent = int(rows.find_all("td")[-2].text)
    # print("Total cases:", total)   
    # print("Recent cases:", recent)

    # remove first 3 and last 2 td tags
    rows = rows.find_all("td")[3:-2]

    # The remaining td tags are sorted as such:
    # Tags containing numbers are the number of cases of each area in the cluster
    # Tags containing text are the names of the areas in the cluster
    # We want to extract the names of the areas and the number of cases
    areas = []
    cases = []
    for i in range(len(rows)):
        if i % 2 == 0:
            areas.append(rows[i].text)
        else:
            cases.append(int(rows[i].text))
    # print("Areas:", areas)
    # print("Cases:", cases)

    # Get the date and month number
    date = snapshots[snapshot_no][:8]
    date = pd.to_datetime(date, format="%Y/%m/%d")
    month = date.month
    # print("Date:", date)
    # print("Month:", month, "\n")

    # Create a dictionary to store the data
    data = {"Number of Cases": cases, "Street Address": areas, "Cluster Number": cluster_no, "Recent Cases in Cluster": recent, "Total Cases in Cluster": total, "Date": date, "Month Number": month}
    df = pd.DataFrame(data)
    df = df.explode("Street Address")
    return df

In [6]:

def extract_snapshot(snapshot):
    # Extract the data from the first snapshot
    soup = get_soup(link, snapshots[snapshot])
    clusters, table = extract_clusters(soup)

    # Create a dataframe to store the data
    df = pd.DataFrame(columns=["Number of Cases", "Street Address", "Cluster Number", "Recent Cases in Cluster", "Total Cases in Cluster", "Date", "Month Number"])

    # from 1 to the number of clusters, keep in mind NEA is stupid sometimes, there may be skipped cluster numbers
    startindex = 1
    endindex = clusters + 1
    while True:
        try:
            # Use concat to append the dataframes
            df = pd.concat([df, extract_cluster(startindex, table, snapshot)], ignore_index=True)
            startindex += 1            
        except:
            if startindex == endindex:
                break
            else:
                startindex += 1
                endindex += 1
                continue           

    return df

In [7]:
# Extract the data from all snapshots
df = pd.DataFrame(columns=["Number of Cases", "Street Address", "Cluster Number", "Recent Cases in Cluster", "Total Cases in Cluster", "Date", "Month Number"])
start = time.time()

# Use concurrent.futures to speed up the process
with ThreadPoolExecutor() as executor:
    results = [executor.submit(extract_snapshot, snapshot) for snapshot in range(len(snapshots))]

    for i in as_completed(results):
        df = pd.concat([df, i.result()], ignore_index=True)

# Change data format to slashes and change street address to lowercase
df["Date"] = df["Date"].dt.strftime("%d/%m/%Y")
df["Street Address"] = df["Street Address"].str.lower()

print("Total Time taken:", format(time.time() - start, ".2f"), "seconds")
df

Total Time taken: 85.10 seconds


,Number of Cases,Street Address,Cluster Number,Recent Cases in Cluster,Total Cases in Cluster,Date,Month Number
0,1,da silva lane,1,2,221,03/12/2021,12
1,28,florence road,1,2,221,03/12/2021,12
2,48,hougang avenue 2,1,2,221,03/12/2021,12
3,2,hougang avenue 2(blk 701),1,2,221,03/12/2021,12
4,2,hougang avenue 2(blk 702),1,2,221,03/12/2021,12
...,...,...,...,...,...,...,...
19699,1,woodlands street 13(blk 104),399,1,2,18/06/2022,6
19700,1,yishun avenue 6(blk 284),400,2,2,18/06/2022,6
19701,1,yishun street 22(blk 278),400,2,2,18/06/2022,6
19702,1,yung kuang road(blk 165b),401,1,2,18/06/2022,6


In [8]:
# Save progress
df.to_csv("dengue2.csv", index=False)

In [9]:
%%time
dir_path = os.getcwd() + "\\"

def set_options(xl, option):
    try:
        xl.Visible = option
        xl.ScreenUpdating = option
        xl.DisplayAlerts = option
        xl.EnableEvents = option
    except: 
        pass

def convert(file):
    file = os.path.splitext(file)[0]
    print('Converting {}...'.format(file))
    xl=win32com.client.Dispatch("Excel.Application",pythoncom.CoInitialize())
    set_options(xl, False)
    
    # check if file with .csv exists
    if not os.path.isfile(dir_path + file + '.csv'):
        try:
            wb = xl.Workbooks.Open(Filename= dir_path + file + '.xlsx',ReadOnly=1)
        except:
            wb = xl.Workbooks.Open(Filename= dir_path + file + '.xlsb',ReadOnly=1)  
        xl.ActiveWorkbook == wb
        
        #6 means csv
        wb.SaveAs(Filename= dir_path + file + '.csv', FileFormat='6') 
        set_options(xl, True)
        wb.Close(True)
        xl.Application.Quit()
        wb=xl=None
    
    df = pd.read_csv(dir_path + file + '.csv', low_memory = False, encoding='ISO-8859-1')
    df.to_pickle(dir_path + file + '.pkl')
    
    return file + '.pkl'

def read(i):
    try: 
        pkl = glob.glob(os.path.join('*{0}*.pkl'.format(i)))[0]
    except: 
        pkl = convert(glob.glob(os.path.join('*{0}*'.format(i)))[0])
    df = pd.read_pickle(dir_path + pkl)
    return df

CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
df = read("dengue2")
# Based on Singapore street address, we can extract the latitude and longitude of the street address
# Enhanced cleaning if the address isn't a block number but still contains brackets
def clean_address(address, original_address):
    if "blk" not in original_address and "(" in original_address and ")" in original_address:
        address = original_address.split("(")[1].replace(")", "")
    elif "#" in address:
        address = address.replace("#", "")
    elif "blk" in original_address:
        #Shift only number after blk to front of address and remove number behind address
        address = original_address.split("blk")[-1].replace(")","") + " " + " ".join(address.split(" ")[:-1])
    return address

def prelim_clean(address):
    trash = [[")", ""],["(blk ", " "],["(", " "]]
    for i in trash:
        address = address.replace(i[0], i[1])
    return address

def request_lat_long(address):
    # Get the latitude and longitude of the address
    req = "https://developers.onemap.sg/commonapi/search?searchVal={address}&returnGeom=Y&getAddrDetails=N&pageNum=1"
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }
    return requests.get(req.format(address=address), headers=headers).json()

def get_lat_long(address):
    trash = [[")", ""],["(blk ", " "],["(", " "]]
    address2 = prelim_clean(address)
    for i in trash:
        address2 = address2.replace(i[0], i[1])

    # Request might time out, so we need to try again
    counter = 0
    while True:
        try:
            if counter == 1:
                address2 = clean_address(address2, address)
            elif counter == 2:
                # print("Address not found:", address)
                return "", ""
            
            response = request_lat_long(address2)
            if len(response['results'])>0:
                return response['results'][0]['LATITUDE'], response['results'][0]['LONGITUDE']
            else:
                counter += 1
        except Exception as e:
            counter += 1
            time.sleep(1)
    


# get only unique street addresses as separate dataframe
df_addresses = df[["Street Address"]].drop_duplicates().reset_index(drop=True)
df_addresses.shape

(5210, 1)

In [11]:
# get the latitude and longitude of the street addresses
# Use threading to speed up the process
start = time.time()
with ThreadPoolExecutor() as executor:
    result = executor.map(get_lat_long, df_addresses["Street Address"])
    df_addresses["Latitude"], df_addresses["Longitude"] = zip(*result)
print("Total Time taken:", format(time.time() - start, ".2f"), "seconds")

Total Time taken: 23.54 seconds


In [12]:
# Use swifter in this case it's faster
import swifter
from swifter import set_defaults
set_defaults(
    allow_dask_on_strings=True,
)
from fuzzywuzzy import process

def get_closest_match(address):
    matches = process.extract(address, df_addresses[(df_addresses["Latitude"] != "") & (df_addresses["Longitude"] != "")]["Street Address"], limit=1)
    return matches[0][0]

def retry_get_lat_long(df_addresses):
    # Create separate dataframe for missing latitude and longitude and drop them from the original dataframe
    df_missing = df_addresses[(df_addresses["Latitude"] == "") | (df_addresses["Longitude"] == "")].reset_index(drop=True)
    df_addresses = df_addresses[(df_addresses["Latitude"] != "") & (df_addresses["Longitude"] != "")].reset_index(drop=True)
    print("Missing Latitudes and Longitudes:", df_missing.shape[0])

    # Multiple fallbacks
    df_missing["1st Cleaned Address"] = df_missing["Street Address"].apply(prelim_clean)
    df_missing["2nd Cleaned Address"] = df_missing.apply(lambda x: clean_address(x["1st Cleaned Address"], x["Street Address"]), axis=1)
    df_missing["3rd Cleaned Address"] = df_missing["2nd Cleaned Address"].swifter.apply(get_closest_match)

    # Get the latitude and longitude of the missing street addresses with 2nd fallback, try 5 times
    counter = 0
    while True:
        with ThreadPoolExecutor() as executor:
            result = executor.map(get_lat_long, df_missing["2nd Cleaned Address"])
            df_missing["Latitude"], df_missing["Longitude"] = zip(*result)
        df_addresses = pd.concat([df_addresses, df_missing[(df_missing["Latitude"] != "") & (df_missing["Longitude"] != "")][["Street Address", "Latitude", "Longitude"]]], axis=0).reset_index(drop=True)
        print("Number of addresses fixed:", df_missing[(df_missing["Latitude"] != "") & (df_missing["Longitude"] != "")].shape[0])
        df_missing = df_missing[(df_missing["Latitude"] == "") | (df_missing["Longitude"] == "")].reset_index(drop=True)
        counter += 1
        if df_missing.shape[0] == 0 or counter == 5:
            break

    # Get the latitude and longitude of remaining missing street addresses with 3rd fallback
    df_missing = df_missing[(df_missing["Latitude"] == "") | (df_missing["Longitude"] == "")].reset_index(drop=True).drop(["Latitude", "Longitude", "Street Address"], axis=1)
    df_missing = df_missing.merge(df_addresses, left_on="3rd Cleaned Address", right_on="Street Address", how="left")
    df_addresses = pd.concat([df_addresses, df_missing[(df_missing["Latitude"] != "") & (df_missing["Longitude"] != "")][["Street Address", "Latitude", "Longitude"]]], axis=0).reset_index(drop=True)
    print("Number of addresses fixed:", df_missing[(df_missing["Latitude"] != "") & (df_missing["Longitude"] != "")].shape[0])

    # Are there more missing latitude and longitude?
    print("Missing Latitude and Longitude:", df_addresses[(df_addresses["Latitude"] == "") | (df_addresses["Longitude"] == "")].shape[0])

    return df_addresses

In [13]:
# Retry to get the latitude and longitude of the missing street addresses
start = time.time()
df_addresses = retry_get_lat_long(df_addresses)
print("Total Time taken:", format(time.time() - start, ".2f"), "seconds")
df_addresses

Missing Latitudes and Longitudes: 107


Dask Apply:   0%|          | 0/48 [00:00<?, ?it/s]

c:\Python310\lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


Number of addresses fixed: 1
Number of addresses fixed: 0
Number of addresses fixed: 0
Number of addresses fixed: 0
Number of addresses fixed: 0
Number of addresses fixed: 106
Missing Latitude and Longitude: 0
Total Time taken: 16.36 seconds


c:\Python310\lib\site-packages\swifter\swifter.py:87: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  warnings.warn(


,Street Address,Latitude,Longitude
0,da silva lane,1.36355650607885,103.883935704039
1,florence road,1.36270126451482,103.887027478789
2,hougang avenue 2,1.36868496277618,103.884309433634
3,hougang avenue 2(blk 701),1.36548398291324,103.890200268843
4,hougang avenue 2(blk 702),1.36575849509964,103.889720136349
...,...,...,...
5205,ubi road 1,1.32643266298531,103.893626872544
5206,lorong k telok kurau(the cristallo),1.31266290803793,103.911509004252
5207,grove drive,1.31423284539755,103.783232610158
5208,ang mo kio avenue 1(blk 225),1.36739612776859,103.838150007464


In [14]:
# Merge the latitude and longitude to the original dataframe
df = df.merge(df_addresses, on="Street Address", how="left")
df

,Number of Cases,Street Address,Cluster Number,Recent Cases in Cluster,Total Cases in Cluster,Date,Month Number,Latitude,Longitude
0,1,da silva lane,1,2,221,03/12/2021,12,1.36355650607885,103.883935704039
1,28,florence road,1,2,221,03/12/2021,12,1.36270126451482,103.887027478789
2,28,florence road,1,2,221,03/12/2021,12,1.36270126451482,103.887027478789
3,28,florence road,1,2,221,03/12/2021,12,1.36270126451482,103.887027478789
4,28,florence road,1,2,221,03/12/2021,12,1.36270126451482,103.887027478789
...,...,...,...,...,...,...,...,...,...
20206,1,yishun street 21(blk 208),398,1,2,16/06/2022,6,1.43102233778042,103.837623233094
20207,1,yishun street 71(blk 729),399,1,2,16/06/2022,6,1.42585285618241,103.831598307804
20208,1,yishun street 72(blk 755),399,1,2,16/06/2022,6,1.42707820696356,103.832706088136
20209,1,yung kuang road(blk 165b),400,1,2,16/06/2022,6,1.33064472260389,103.722321863113


In [15]:
# Check for missing values in the dataframe
df.isnull().sum()

Number of Cases              0
Street Address               0
Cluster Number               0
Recent Cases in Cluster      0
Total Cases in Cluster       0
Date                         0
Month Number                 0
Latitude                   301
Longitude                  301
dtype: int64

In [16]:
# Bring the latitude and longitude to column 3 and 4
df = df[["Number of Cases", "Street Address", "Latitude", "Longitude", "Cluster Number", "Recent Cases in Cluster", "Total Cases in Cluster", "Date", "Month Number"]]
# Save progress
df.to_csv("dengue2.csv", index=False)
# Remove pkl
os.remove("dengue2.pkl")

In [17]:
df_1 = read('dengue')
df_2 = read('dengue2')

Converting dengue2...


In [18]:
# Remove duplicate rows from df_2 which are defined by having the same number of cases, cluster number, recent cases in cluster, total cases in cluster, and street address
df_2 = df_2.drop_duplicates(subset=["Number of Cases", "Recent Cases in Cluster", "Total Cases in Cluster", "Street Address"], keep="first").reset_index(drop=True)

# list of unique date
date_list = df_2["Date"].unique()

from datetime import datetime
date_list = sorted(date_list, key=lambda x: datetime.strptime(x, '%d/%m/%Y'))
print(date_list)

new_list = []
for item in date_list:
    if len(new_list) == 0:
        new_list.append(item)
    else:
        if (datetime.strptime(item, '%d/%m/%Y') - datetime.strptime(new_list[-1], '%d/%m/%Y')).days >= 7:
            new_list.append(item) 
print(new_list)

# filter df_2 with better regularity
df_2 = df_2[df_2["Date"].isin(new_list)].reset_index(drop=True)
df_2

['08/03/2021', '08/08/2021', '27/10/2021', '03/12/2021', '10/03/2022', '19/03/2022', '29/03/2022', '25/04/2022', '05/06/2022', '09/06/2022', '16/06/2022', '18/06/2022', '05/08/2022', '14/08/2022', '01/09/2022', '07/09/2022', '11/09/2022', '18/09/2022', '21/09/2022', '25/10/2022']
['08/03/2021', '08/08/2021', '27/10/2021', '03/12/2021', '10/03/2022', '19/03/2022', '29/03/2022', '25/04/2022', '05/06/2022', '16/06/2022', '05/08/2022', '14/08/2022', '01/09/2022', '11/09/2022', '18/09/2022', '25/10/2022']


,Number of Cases,Street Address,Latitude,Longitude,Cluster Number,Recent Cases in Cluster,Total Cases in Cluster,Date,Month Number
0,1,da silva lane,1.363557,103.883936,1,2,221,03/12/2021,12
1,28,florence road,1.362701,103.887027,1,2,221,03/12/2021,12
2,48,hougang avenue 2,1.368685,103.884309,1,2,221,03/12/2021,12
3,2,hougang avenue 2(blk 701),1.365484,103.890200,1,2,221,03/12/2021,12
4,2,hougang avenue 2(blk 702),1.365758,103.889720,1,2,221,03/12/2021,12
...,...,...,...,...,...,...,...,...,...
12892,1,tuas view square,1.299826,103.626004,391,2,2,16/06/2022,6
12893,2,upper serangoon crescent(riversails),1.378120,103.902296,393,2,2,16/06/2022,6
12894,2,west coast way(carabelle),1.318025,103.756289,395,1,2,16/06/2022,6
12895,1,yishun street 71(blk 729),1.425853,103.831598,399,1,2,16/06/2022,6


In [19]:
# Concatenate the two dataframes
df_final = pd.concat([df_1, df_2], ignore_index=True)
df_final

,Number of Cases,Street Address,Latitude,Longitude,Cluster Number,Recent Cases in Cluster,Total Cases in Cluster,Date,Month Number
0,4,bishan street 22 (block 232),1.358286,103.845226,1,24,83,3/7/2015,7
1,7,bishan street 22 (block 233),1.358639,103.845259,1,24,83,3/7/2015,7
2,3,bishan street 22 (block 234),1.358390,103.845955,1,24,83,3/7/2015,7
3,1,bishan street 22 (block 235),1.358719,103.846477,1,24,83,3/7/2015,7
4,4,bishan street 22 (block 236),1.359041,103.846849,1,24,83,3/7/2015,7
...,...,...,...,...,...,...,...,...,...
69868,1,tuas view square,1.299826,103.626004,391,2,2,16/06/2022,6
69869,2,upper serangoon crescent(riversails),1.378120,103.902296,393,2,2,16/06/2022,6
69870,2,west coast way(carabelle),1.318025,103.756289,395,1,2,16/06/2022,6
69871,1,yishun street 71(blk 729),1.425853,103.831598,399,1,2,16/06/2022,6


In [20]:
df_lat_long = df_final[["Latitude", "Longitude"]].drop_duplicates().reset_index(drop=True)

In [21]:
# use master-plan-2019-subzone-boundary-no-sea-geojson.geojson to get the subzone
with open("master-plan-2019-subzone-boundary-no-sea-geojson.geojson") as f:
    geojson = json.load(f)

# Create dataframe of features and the geometry
df_geojson = pd.DataFrame(geojson["features"])
df_geojson["geometry"] = df_geojson["geometry"].apply(lambda x: shape(x))

# extract subzone, planning area, region
def extract_areas(column):
    properties = column["Description"]
    soup = BeautifulSoup(properties, "lxml")
    subzone = soup.find("th", text="SUBZONE_N").find_next_sibling("td").text
    planning_area = soup.find("th", text="PLN_AREA_N").find_next_sibling("td").text
    region = soup.find("th", text="REGION_N").find_next_sibling("td").text
    return subzone, planning_area, region

df_geojson["Subzone"], df_geojson["Planning Area"], df_geojson["Region"] = zip(*df_geojson["properties"].apply(lambda x: extract_areas(x)))
df_geojson.drop(columns=["properties"], inplace=True)


In [22]:
# Convert to GeoDataFrame
gdf_geojson = gpd.GeoDataFrame(df_geojson, geometry="geometry")
gdf_lat_long = gpd.GeoDataFrame(df_lat_long, geometry=gpd.points_from_xy(df_lat_long["Longitude"], df_lat_long["Latitude"]))
print("Converted to GeoDataFrame.")

# Spatial join
gdf_lat_long = gpd.sjoin(gdf_lat_long, gdf_geojson, how="left", predicate="within")
gdf_lat_long.drop(columns=["index_right"], inplace=True)
print("Spatial join done.")

Converted to GeoDataFrame.
Spatial join done.


In [23]:
# # G_MP19_LAND_USE_PL.kml is used to get the land use of each polygon
# # Convert it to geojson via kml2geojson
# import kml2geojson

# geojson = kml2geojson.main.convert("G_MP19_LAND_USE_PL.kml")
# with open("G_MP19_LAND_USE_PL.geojson", "w") as f:
#     json.dump(geojson, f)

In [24]:
# use G_MP19_LAND_USE_PL.geojson to get the land use
with open("G_MP19_LAND_USE_PL.geojson") as f:
    geojson = json.load(f)[0]

# Create dataframe of features and the geometry
df_features = pd.DataFrame(geojson["features"])
df_features["geometry"] = df_features["geometry"].apply(lambda x: shape(x))
df_features.drop(columns=["id", "type"], inplace=True)

df_features

,properties,geometry
0,"{'name': 'kml_1', 'description': '<center><tab...",POLYGON Z ((103.816575999563 1.33889054098461 ...
1,"{'name': 'kml_2', 'description': '<center><tab...",POLYGON Z ((103.835331928443 1.34170755577232 ...
2,"{'name': 'kml_3', 'description': '<center><tab...","POLYGON Z ((103.73497338744 1.3761362564348 0,..."
3,"{'name': 'kml_4', 'description': '<center><tab...",POLYGON Z ((103.866674390378 1.40520212404975 ...
4,"{'name': 'kml_5', 'description': '<center><tab...",POLYGON Z ((103.868548430614 1.40994869694386 ...
...,...,...
113207,"{'name': 'kml_113208', 'description': '<center...",POLYGON Z ((103.850612536182 1.30173137826202 ...
113208,"{'name': 'kml_113209', 'description': '<center...",POLYGON Z ((103.952518503545 1.32955163912673 ...
113209,"{'name': 'kml_113210', 'description': '<center...",POLYGON Z ((103.967238574619 1.34540563074645 ...
113210,"{'name': 'kml_113211', 'description': '<center...",POLYGON Z ((103.96274641536 1.31711238061229 0...


In [25]:
def extract_land_use(cell):
    properties = cell["description"]
    soup = BeautifulSoup(properties, "lxml")
    land_use = soup.find("th", text="LU_DESC").find_next_sibling("td").text
    return land_use

# Use swifter in this case it's faster
start = time.time()
import swifter
from swifter import set_defaults
set_defaults(
    allow_dask_on_strings=True,
)

df_features["Land Use"] = df_features["properties"].swifter.apply(lambda x: extract_land_use(x))
df_features.drop(columns=["properties"], inplace=True)
print("Total Time taken:", format(time.time() - start, ".2f"), "seconds")

Dask Apply:   0%|          | 0/48 [00:00<?, ?it/s]

Total Time taken: 12.34 seconds


In [26]:
# Convert both dataframes to geodataframes
gdf_features = gpd.GeoDataFrame(df_features, geometry=df_features["geometry"])
print("Converted to geodataframes.")

# Spatial join to get the land use of each point
gdf_lat_long_land_use = gpd.sjoin(gdf_lat_long, gdf_features, how="left", op="within")
gdf_lat_long_land_use.drop(columns=["index_right", "geometry", "type"], inplace=True)
print("Spatial join done.")

# Merge the land use to the original dataframe
df_final = df_final.merge(gdf_lat_long_land_use, how="left", on=["Latitude", "Longitude"])
df_final

Converted to geodataframes.
Spatial join done.


,Number of Cases,Street Address,Latitude,Longitude,Cluster Number,Recent Cases in Cluster,Total Cases in Cluster,Date,Month Number,Subzone,Planning Area,Region,Land Use
0,4,bishan street 22 (block 232),1.358286,103.845226,1,24,83,3/7/2015,7,MARYMOUNT,BISHAN,CENTRAL REGION,RESIDENTIAL
1,7,bishan street 22 (block 233),1.358639,103.845259,1,24,83,3/7/2015,7,MARYMOUNT,BISHAN,CENTRAL REGION,RESIDENTIAL
2,3,bishan street 22 (block 234),1.358390,103.845955,1,24,83,3/7/2015,7,MARYMOUNT,BISHAN,CENTRAL REGION,RESIDENTIAL
3,1,bishan street 22 (block 235),1.358719,103.846477,1,24,83,3/7/2015,7,MARYMOUNT,BISHAN,CENTRAL REGION,RESIDENTIAL
4,4,bishan street 22 (block 236),1.359041,103.846849,1,24,83,3/7/2015,7,MARYMOUNT,BISHAN,CENTRAL REGION,RESIDENTIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69868,1,tuas view square,1.299826,103.626004,391,2,2,16/06/2022,6,TUAS VIEW,TUAS,WEST REGION,BUSINESS 2
69869,2,upper serangoon crescent(riversails),1.378120,103.902296,393,2,2,16/06/2022,6,HOUGANG EAST,HOUGANG,NORTH-EAST REGION,RESIDENTIAL
69870,2,west coast way(carabelle),1.318025,103.756289,395,1,2,16/06/2022,6,WEST COAST,CLEMENTI,WEST REGION,RESIDENTIAL
69871,1,yishun street 71(blk 729),1.425853,103.831598,399,1,2,16/06/2022,6,YISHUN WEST,YISHUN,NORTH REGION,RESIDENTIAL WITH COMMERCIAL AT 1ST STOREY


In [27]:
# Save
df_final.to_csv("dengue_final.csv", index=False)

In [28]:
# Make a authentication post request to Onemap Singapore API
from dotenv import load_dotenv
load_dotenv()

email = os.getenv("EMAIL")
password = os.getenv("PASSWORD")
token = os.getenv("TOKEN")

def save_token():
    params = {
    "email": email,
    "password": password
    }

    r = requests.post("https://developers.onemap.sg/privateapi/auth/post/getToken", json=params)
    token = r.json()["access_token"]

    with open(".env", "a") as f:
        f.write("TOKEN={0}".format(token))
    print("Token updated:", token)

    return token


# Get list of planning areas
try: 
    r = requests.get("https://developers.onemap.sg/privateapi/popapi/getPlanningareaNames?token={0}".format(token))
except: 
    token = save_token()
    r = requests.get("https://developers.onemap.sg/privateapi/popapi/getPlanningareaNames?token={0}".format(token))

planning_areas = [item["pln_area_n"] for item in r.json()]


# save into dataframe
df_population = pd.DataFrame(planning_areas, columns=["Planning Area"])
df_population.head()

,Planning Area
0,ANG MO KIO
1,BEDOK
2,BISHAN
3,BOON LAY
4,BUKIT BATOK


In [29]:
# For each planning area, get the population
def get_population(year, planning_area):

    # There is an api limit of 250 per minute
    while True:
        try:
            r = requests.get("https://developers.onemap.sg/privateapi/popapi/getPopulationAgeGroup?token={0}&year={1}&planningArea={2}".format(token, year, planning_area))
            # json contains 3 responses, one for each gender type
            # we only need the total population which is usually the second response, but we will check just in case 
            if r.json()[1]["gender"] == "Total":
                return r.json()[1]
            elif r.json()[2]["gender"] == "Total":
                return r.json()[2]
            else:
                return r.json()[0]
        except:
            time.sleep(1)
            save_token()

    

In [30]:
# Test for row 1
test_columns = get_population(2019, df_population["Planning Area"][0])

# We will have each age category as a column as well as the total population
columns = [item for item in test_columns.keys() if item.startswith("age") or item == "total"]
for column in columns:
    df_population[column] = 0

with ThreadPoolExecutor() as executor:
    results = executor.map(get_population, [2019] * len(df_population), df_population["Planning Area"])
    df_population["json_2019"] = list(results)
    
# Then we will populate the columns
for column in columns:
    df_population[column] = df_population.apply(lambda x: x["json_2019"][column], axis=1)


In [31]:
#move json to 3rd dataframe
df_population_all = df_population[["Planning Area", "json_2019"]].copy()
df_population.drop("json_2019", axis=1, inplace=True)

# Save
df_population.to_csv("population_2019.csv", index=False)

In [32]:
# Now we go crazy with all the years except 2019 which we already got
years = [2015, 2016, 2017, 2018, 2020, 2021]

for year in years:
    print("Getting population for year {0}".format(year))
    with ThreadPoolExecutor() as executor:
        results = executor.map(get_population, [year] * len(df_population), df_population["Planning Area"])
        df_population_all["json_{0}".format(year)] = list(results)
    
        
# Save
df_population_all = df_population_all.reindex(sorted(df_population_all.columns), axis=1)
df_population_all.to_csv("population_all.csv", index=False)

Getting population for year 2015
Getting population for year 2016
Getting population for year 2017
Getting population for year 2018
Getting population for year 2020
Getting population for year 2021
